In [1]:
import csv
import math
import string
import os

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from PIL import Image
from IPython.display import display, Markdown

from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn import svm
from sklearn import preprocessing
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.neighbors import NearestNeighbors as KNN
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import Lasso
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import Perceptron
from sklearn.linear_model import Ridge
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.kernel_ridge import KernelRidge
from sklearn.gaussian_process.kernels import RBF

from joblib import dump, load

from nltk.stem import PorterStemmer
import nltk
from nltk.corpus import stopwords

import gensim

import torch
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import torch.utils.data as Data
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM
from torch.autograd import Variable

import utils.BoW_data as dp
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gsuri\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
ddir = 'data/'

In [3]:
mlb_tags_train, mlb_tags_test, df = dp.load_dataframe()

In [4]:
# Have to do this because load_dataframe() is messing up resnet_vectors somehow
with open('data/features_train/features_resnet1000_train.csv', 'r') as fp:
    features_train = [x.strip().split(',') for x in fp.readlines()]
    
features_train = {x[0]: np.array([np.float(n) for n in x[1:]]) for x in features_train}

with open('data/features_test/features_resnet1000_test.csv', 'r') as fp:
    features_test = [x.strip().split(',') for x in fp.readlines()]
    
features_test = {x[0]: np.array([np.float64(n) for n in x[1:]]) for x in features_test}

In [5]:
# Have to do this because load_dataframe() is messing up resnet_vectors somehow
with open('data/features_train/features_resnet1000intermediate_train.csv', 'r') as fp:
    int_features_train = [x.strip().split(',') for x in fp.readlines()]
    
int_features_train = {x[0]: np.array([np.float(n) for n in x[1:]]) for x in int_features_train}

# Have to do this because load_dataframe() is messing up resnet_vectors somehow
with open('data/features_test/features_resnet1000intermediate_test.csv', 'r') as fp:
    int_features_test = [x.strip().split(',') for x in fp.readlines()]
    
int_features_test = {x[0]: np.array([np.float64(n) for n in x[1:]]) for x in int_features_test}

# W2V

In [6]:
df['desc_joined'] = df['descriptions'].apply(lambda x: " ".join(x).split(" "))

In [7]:
# w2v_model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)
#Comment this out and use above line if you already have word2vec google stuff downloaded
import gensim.downloader as api

w2v_model = api.load("word2vec-google-news-300")

In [8]:
ps = PorterStemmer()

stop_words = set(stopwords.words('english'))
stop_words.remove('not')

In [9]:
def get_word2vec(df_in, model):
    
    def prep_doc(doc):

        # Stem and lower all the words
        sentence = [
            ps.stem(word).lower() for word in doc
        ]

        # Remove trailing commas and periods
        sentence = [
            "".join([char for char in word if char not in string.punctuation]) for word in sentence
        ]

        # Remove stopwords, remove any blank words 
        sentence = [
            word for word in sentence if word not in stop_words and word not in  [" ", ""]
        ]
        
        return sentence
    
    def try_get_vec(word):
        try:
            word = model.get_vector(word)
            can_do = True
        except:
            word = np.zeros(300)
            can_do = False
        return can_do
    
    def get_vector(word):
        try:
            word = model.get_vector(word)
        except:
            word = np.zeros(300)
        return word
    
    stops = set(stopwords.words('english'))
    vecs = df_in['desc_joined'].apply(
        lambda desc: np.average([
            get_vector(word) for word in prep_doc(desc)
            if try_get_vec(word)
        ], axis=0)
    )
    
    return np.asarray([vec for vec in vecs], dtype=np.float64)

# BERT

In [10]:
# Return train and test dicts for looking up BERT vector from filename

bert_train = torch.load(ddir + 'bert/bert_train_avg_last_4.pt')
bert_test = torch.load(ddir + 'bert/bert_test_avg_last_4.pt')

for root, directory, files in os.walk(os.path.join('data/','descriptions_train')):
    bert_train_filenames = files
    
for root, directory, files in os.walk(os.path.join('data/','descriptions_test')):
    bert_test_filenames = files
    
bert_train_lookup = {bert_train_filenames[ind]: bert_vector.numpy() for ind, bert_vector in enumerate(bert_train)}
bert_test_lookup = {bert_test_filenames[ind]: bert_vector.numpy() for ind, bert_vector in enumerate(bert_test)}

In [10]:
df['text_file'] = df['image_file'].apply(lambda x: x.split('/')[-1].replace('jpg', 'txt'))
df.head()

,image_file,resnet_vector,descriptions,tags,tags_values,tags_vec,word_list,word_vector,desc_joined,text_file
0,images_train/5373.jpg,"[-0.8994496464729309, -0.9304700493812561, -2....","[a red train is docked at the station, Several...","[vehicle:train, person:person, indoor:clock, a...","[train, person, clock, handbag]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[red, train, dock, station, sever, peopl, stan...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[a, red, train, is, docked, at, the, station, ...",5373.txt
1,images_train/984.jpg,"[-1.3469539880752563, -3.1194605827331543, -0....",[A man with blue jersey holding a baseball bat...,"[person:person, sports:baseball bat]","[person, baseball bat]","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[A, man, blue, jersey, hold, basebal, bat, clo...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[A, man, with, blue, jersey, holding, a, baseb...",984.txt
2,images_train/7127.jpg,"[-3.44549822807312, -1.5245732069015503, -1.00...",[A kitchen decorated in red and white with acc...,"[appliance:refrigerator, appliance:oven, appli...","[refriger, oven, sink, cup, cake, vase]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[A, kitchen, decor, red, white, accessori, A, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[A, kitchen, decorated, in, red, and, white, w...",7127.txt
3,images_train/9609.jpg,"[1.1146496534347534, -2.1671018600463867, 0.09...",[A black and white dog chasing sheep in a fiel...,"[animal:dog, animal:sheep]","[dog, sheep]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[A, black, white, dog, chase, sheep, field, sm...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[A, black, and, white, dog, chasing, sheep, in...",9609.txt
4,images_train/5293.jpg,"[1.6026496887207031, -1.5058174133300781, 3.02...",[Two bears with their mouths open in the water...,[animal:bear],[bear],"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[two, bear, mouth, open, water, A, coupl, bear...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[Two, bears, with, their, mouths, open, in, th...",5293.txt


# Training / test data prep

In [41]:
# X = np.asarray([bert_train_lookup[file] for file in df['text_file']], dtype=np.float64)
X = get_word2vec(df, w2v_model)
# y = df['resnet_vector'].to_numpy()
y = np.asarray([int_features_train[x] for x in df['image_file']], dtype=np.float64)

y_true = df['image_file'].to_numpy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)


(8000, 300)
(8000, 2048)
(2000, 300)
(2000, 2048)


In [20]:
y_final = np.asarray([features_train[x] for x in df['image_file']], dtype=np.float64)

X_train_final, X_test_final, y_train_final, y_test_final = train_test_split(X, y_final, test_size=0.2)

# KNN Classifier

In [12]:
neigh = KNN(n_neighbors=3)
clf.fit(X_train, y_train)

NameError: name 'clf' is not defined

# Random Forest Regressor

In [13]:
rfr = RandomForestRegressor(max_depth=3, random_state=0, n_estimators=20)
rfr.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=3,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=20,
                      n_jobs=None, oob_score=False, random_state=0, verbose=0,
                      warm_start=False)

In [14]:
rfr.score(X_test, y_test)

C:\Users\gsuri\AppData\Local\Continuum\anaconda3\envs\Python3.7\lib\site-packages\sklearn\base.py:420: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)


0.10931067331754522

# Lasso Regression

In [15]:
clf = Lasso(alpha=0.1)
clf.fit(X_train, y_train)

Lasso(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

In [16]:
clf.score(X_test, y_test)

C:\Users\gsuri\AppData\Local\Continuum\anaconda3\envs\Python3.7\lib\site-packages\sklearn\base.py:420: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)


-0.0005410225173550954

# Ridge Regression

In [21]:
clf = Ridge(alpha=150)
clf.fit(X_train, y_train)

Ridge(alpha=150, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)

In [22]:
clf.score(X_test, y_test)

C:\Users\gsuri\AppData\Local\Continuum\anaconda3\envs\Python3.7\lib\site-packages\sklearn\base.py:420: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)


0.2422885007709747

# Kernel Ridge

In [42]:
yyy = np.zeros(8000)

In [43]:
clf = KernelRidge()
clf.fit(X_train, y_train)

KernelRidge(alpha=1, coef0=1, degree=3, gamma=None, kernel='linear',
            kernel_params=None)

In [44]:
clf.score(X_test, y_test)

C:\Users\gsuri\AppData\Local\Continuum\anaconda3\envs\Python3.7\lib\site-packages\sklearn\base.py:420: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)


0.33310892114748325

# Even Kernellyer Ridge

In [22]:
clf = KernelRidge(kernel=RBF(1))
clf.fit(X_train, y_train)

KernelRidge(alpha=1, coef0=1, degree=3, gamma=None, kernel=RBF(length_scale=1),
            kernel_params=None)

In [78]:
clf.score(X_test, y_test)

/usr/local/lib/python3.7/site-packages/sklearn/base.py:420: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)


0.3740517965572554

# Linear Regression

In [23]:
clf = LinearRegression()
clf.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [24]:
clf.score(X_test, y_test)

C:\Users\gsuri\AppData\Local\Continuum\anaconda3\envs\Python3.7\lib\site-packages\sklearn\base.py:420: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)


0.334354686751342

# SKLearn NN

At first tried with final layer at 512 but it was not converging, probably due to compression of the high dimensionality output vector which is 1000

Using final feature vecs with layers [2048, 1024] => 0.41

Need to try with intermediate feature vector, shouldnt be too hard to pull out

In [53]:
solvers = ['lbfgs', 'sgd', 'adam']

nn = MLPRegressor(
    hidden_layer_sizes=(512, 1024, 2048),
    max_iter=300,
    solver='adam',
    activation='relu',
    verbose=True
)

In [55]:
nn2 = MLPRegressor(
    hidden_layer_sizes=(1000, 1024, 512, 300),
    max_iter=800,
    solver='adam',
    activation='relu',
    verbose=True
)

In [56]:
nn2.fit(X_train_final, y_train_final)

Iteration 1, loss = 2.04522225
Iteration 2, loss = 1.54958775
Iteration 3, loss = 1.35353751
Iteration 4, loss = 1.25212135
Iteration 5, loss = 1.19933618
Iteration 6, loss = 1.15881870
Iteration 7, loss = 1.12394853
Iteration 8, loss = 1.09089850
Iteration 9, loss = 1.06167237
Iteration 10, loss = 1.04309686
Iteration 11, loss = 1.02508468
Iteration 12, loss = 1.00933504
Iteration 13, loss = 0.99695030
Iteration 14, loss = 0.98137426
Iteration 15, loss = 0.96275227
Iteration 16, loss = 0.95118901
Iteration 17, loss = 0.93852335
Iteration 18, loss = 0.92267711
Iteration 19, loss = 0.91621295
Iteration 20, loss = 0.90924283
Iteration 21, loss = 0.89371082
Iteration 22, loss = 0.88612982
Iteration 23, loss = 0.87276792
Iteration 24, loss = 0.86569923
Iteration 25, loss = 0.85672859
Iteration 26, loss = 0.84429697
Iteration 27, loss = 0.83683222
Iteration 28, loss = 0.82903991
Iteration 29, loss = 0.82271808
Iteration 30, loss = 0.81474231
Iteration 31, loss = 0.80257218
Iteration 32, los

Iteration 253, loss = 0.24991894
Iteration 254, loss = 0.25162617
Iteration 255, loss = 0.25061743
Iteration 256, loss = 0.25196444
Iteration 257, loss = 0.25241587
Iteration 258, loss = 0.25190126
Iteration 259, loss = 0.25147111
Iteration 260, loss = 0.24706905
Iteration 261, loss = 0.24493883
Iteration 262, loss = 0.24450199
Iteration 263, loss = 0.24395449
Iteration 264, loss = 0.24350467
Iteration 265, loss = 0.24307632
Iteration 266, loss = 0.24567343
Iteration 267, loss = 0.24462492
Iteration 268, loss = 0.24510962
Iteration 269, loss = 0.24199087
Iteration 270, loss = 0.24165116
Iteration 271, loss = 0.24373463
Iteration 272, loss = 0.24349178
Iteration 273, loss = 0.24137592
Iteration 274, loss = 0.24127156
Iteration 275, loss = 0.24186251
Iteration 276, loss = 0.23884622
Iteration 277, loss = 0.23682698
Iteration 278, loss = 0.23477781
Iteration 279, loss = 0.23305193
Iteration 280, loss = 0.23550943
Iteration 281, loss = 0.23533017
Iteration 282, loss = 0.23282685
Iteration 

MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
             beta_2=0.999, early_stopping=False, epsilon=1e-08,
             hidden_layer_sizes=(1000, 1024, 512, 300),
             learning_rate='constant', learning_rate_init=0.001, max_iter=800,
             momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
             power_t=0.5, random_state=None, shuffle=True, solver='adam',
             tol=0.0001, validation_fraction=0.1, verbose=True,
             warm_start=False)

In [57]:
nn.fit(X_train, y_train)

Iteration 1, loss = 0.10102426
Iteration 2, loss = 0.08145103
Iteration 3, loss = 0.07122459
Iteration 4, loss = 0.06679408
Iteration 5, loss = 0.06364042
Iteration 6, loss = 0.06166818
Iteration 7, loss = 0.06025221
Iteration 8, loss = 0.05898217
Iteration 9, loss = 0.05807633
Iteration 10, loss = 0.05721732
Iteration 11, loss = 0.05637248
Iteration 12, loss = 0.05577347
Iteration 13, loss = 0.05504982
Iteration 14, loss = 0.05456464
Iteration 15, loss = 0.05402853
Iteration 16, loss = 0.05338264
Iteration 17, loss = 0.05289800
Iteration 18, loss = 0.05231633
Iteration 19, loss = 0.05184993
Iteration 20, loss = 0.05136230
Iteration 21, loss = 0.05088905
Iteration 22, loss = 0.05042346
Iteration 23, loss = 0.04993417
Iteration 24, loss = 0.04946806
Iteration 25, loss = 0.04894746
Iteration 26, loss = 0.04863802
Iteration 27, loss = 0.04814480
Iteration 28, loss = 0.04763991
Iteration 29, loss = 0.04733494
Iteration 30, loss = 0.04689207
Iteration 31, loss = 0.04644548
Iteration 32, los

MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
             beta_2=0.999, early_stopping=False, epsilon=1e-08,
             hidden_layer_sizes=(512, 1024, 2048), learning_rate='constant',
             learning_rate_init=0.001, max_iter=300, momentum=0.9,
             n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
             random_state=None, shuffle=True, solver='adam', tol=0.0001,
             validation_fraction=0.1, verbose=True, warm_start=False)

In [58]:
cvnn = cross_val_score(
    nn,
    X,
    y, 
    cv=3
)

Iteration 1, loss = 0.10220454
Iteration 2, loss = 0.08385694
Iteration 3, loss = 0.07414783
Iteration 4, loss = 0.06902523
Iteration 5, loss = 0.06584340
Iteration 6, loss = 0.06329180
Iteration 7, loss = 0.06158166
Iteration 8, loss = 0.06005001
Iteration 9, loss = 0.05890663
Iteration 10, loss = 0.05796911
Iteration 11, loss = 0.05710024
Iteration 12, loss = 0.05633342
Iteration 13, loss = 0.05564370
Iteration 14, loss = 0.05503360
Iteration 15, loss = 0.05444797
Iteration 16, loss = 0.05379541
Iteration 17, loss = 0.05324871
Iteration 18, loss = 0.05267100
Iteration 19, loss = 0.05213124
Iteration 20, loss = 0.05166200
Iteration 21, loss = 0.05115623
Iteration 22, loss = 0.05055468
Iteration 23, loss = 0.05007616
Iteration 24, loss = 0.04964900
Iteration 25, loss = 0.04915265
Iteration 26, loss = 0.04862405
Iteration 27, loss = 0.04827774
Iteration 28, loss = 0.04790205
Iteration 29, loss = 0.04739861
Iteration 30, loss = 0.04713990
Iteration 31, loss = 0.04655096
Iteration 32, los

C:\Users\gsuri\AppData\Local\Continuum\anaconda3\envs\Python3.7\lib\site-packages\sklearn\base.py:420: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)


Iteration 1, loss = 0.10215231
Iteration 2, loss = 0.08292071
Iteration 3, loss = 0.07358728
Iteration 4, loss = 0.06864840
Iteration 5, loss = 0.06523792
Iteration 6, loss = 0.06313031
Iteration 7, loss = 0.06126243
Iteration 8, loss = 0.06009119
Iteration 9, loss = 0.05874441
Iteration 10, loss = 0.05775496
Iteration 11, loss = 0.05707055
Iteration 12, loss = 0.05646024
Iteration 13, loss = 0.05561879
Iteration 14, loss = 0.05490768
Iteration 15, loss = 0.05434677
Iteration 16, loss = 0.05381707
Iteration 17, loss = 0.05309559
Iteration 18, loss = 0.05255902
Iteration 19, loss = 0.05204428
Iteration 20, loss = 0.05153042
Iteration 21, loss = 0.05108455
Iteration 22, loss = 0.05042347
Iteration 23, loss = 0.04987197
Iteration 24, loss = 0.04942220
Iteration 25, loss = 0.04909447
Iteration 26, loss = 0.04850616
Iteration 27, loss = 0.04810362
Iteration 28, loss = 0.04757412
Iteration 29, loss = 0.04715899
Iteration 30, loss = 0.04688010
Iteration 31, loss = 0.04638484
Iteration 32, los

C:\Users\gsuri\AppData\Local\Continuum\anaconda3\envs\Python3.7\lib\site-packages\sklearn\base.py:420: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)


Iteration 1, loss = 0.10156083
Iteration 2, loss = 0.08263643
Iteration 3, loss = 0.07294039
Iteration 4, loss = 0.06817724
Iteration 5, loss = 0.06452472
Iteration 6, loss = 0.06226191
Iteration 7, loss = 0.06087026
Iteration 8, loss = 0.05937421
Iteration 9, loss = 0.05834791
Iteration 10, loss = 0.05749729
Iteration 11, loss = 0.05660137
Iteration 12, loss = 0.05571778
Iteration 13, loss = 0.05507044
Iteration 14, loss = 0.05452862
Iteration 15, loss = 0.05394909
Iteration 16, loss = 0.05356731
Iteration 17, loss = 0.05280485
Iteration 18, loss = 0.05218540
Iteration 19, loss = 0.05172049
Iteration 20, loss = 0.05125733
Iteration 21, loss = 0.05070333
Iteration 22, loss = 0.05028975
Iteration 23, loss = 0.04969899
Iteration 24, loss = 0.04920342
Iteration 25, loss = 0.04874440
Iteration 26, loss = 0.04820188
Iteration 27, loss = 0.04793698
Iteration 28, loss = 0.04752334
Iteration 29, loss = 0.04709485
Iteration 30, loss = 0.04654258
Iteration 31, loss = 0.04617019
Iteration 32, los

C:\Users\gsuri\AppData\Local\Continuum\anaconda3\envs\Python3.7\lib\site-packages\sklearn\base.py:420: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)


In [59]:
cvnn

array([0.07628438, 0.06527372, 0.05843292])

In [60]:
cvnn = cross_val_score(
    nn2,
    X,
    y, 
    cv=3
)

Iteration 1, loss = 0.10827620
Iteration 2, loss = 0.08751864
Iteration 3, loss = 0.08195174
Iteration 4, loss = 0.07528029
Iteration 5, loss = 0.07107472
Iteration 6, loss = 0.06912873
Iteration 7, loss = 0.06747279
Iteration 8, loss = 0.06567611
Iteration 9, loss = 0.06426368
Iteration 10, loss = 0.06302664
Iteration 11, loss = 0.06181401
Iteration 12, loss = 0.06078230
Iteration 13, loss = 0.06000873
Iteration 14, loss = 0.05930888
Iteration 15, loss = 0.05875586
Iteration 16, loss = 0.05780155
Iteration 17, loss = 0.05741816
Iteration 18, loss = 0.05675796
Iteration 19, loss = 0.05617448
Iteration 20, loss = 0.05567980
Iteration 21, loss = 0.05507565
Iteration 22, loss = 0.05461222
Iteration 23, loss = 0.05426637
Iteration 24, loss = 0.05384905
Iteration 25, loss = 0.05339910
Iteration 26, loss = 0.05293943
Iteration 27, loss = 0.05259330
Iteration 28, loss = 0.05224870
Iteration 29, loss = 0.05186782
Iteration 30, loss = 0.05134927
Iteration 31, loss = 0.05099853
Iteration 32, los

C:\Users\gsuri\AppData\Local\Continuum\anaconda3\envs\Python3.7\lib\site-packages\sklearn\base.py:420: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)


Iteration 1, loss = 0.10808207
Iteration 2, loss = 0.08755023
Iteration 3, loss = 0.08075068
Iteration 4, loss = 0.07418681
Iteration 5, loss = 0.07090960
Iteration 6, loss = 0.06876197
Iteration 7, loss = 0.06668966
Iteration 8, loss = 0.06502365
Iteration 9, loss = 0.06377520
Iteration 10, loss = 0.06250995
Iteration 11, loss = 0.06150926
Iteration 12, loss = 0.06061374
Iteration 13, loss = 0.05992359
Iteration 14, loss = 0.05899274
Iteration 15, loss = 0.05814460
Iteration 16, loss = 0.05753006
Iteration 17, loss = 0.05679025
Iteration 18, loss = 0.05626793
Iteration 19, loss = 0.05575988
Iteration 20, loss = 0.05522714
Iteration 21, loss = 0.05468040
Iteration 22, loss = 0.05439359
Iteration 23, loss = 0.05393183
Iteration 24, loss = 0.05334447
Iteration 25, loss = 0.05287804
Iteration 26, loss = 0.05248345
Iteration 27, loss = 0.05220054
Iteration 28, loss = 0.05173950
Iteration 29, loss = 0.05131164
Iteration 30, loss = 0.05096598
Iteration 31, loss = 0.05070207
Iteration 32, los

C:\Users\gsuri\AppData\Local\Continuum\anaconda3\envs\Python3.7\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:568: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")
C:\Users\gsuri\AppData\Local\Continuum\anaconda3\envs\Python3.7\lib\site-packages\sklearn\base.py:420: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)
C:\Users\gsuri\AppData\Local\Continuum\anaconda3\envs\Python3.7\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:568: UserWarning: Training interrupted by user.
  warnings.warn("Training inte

In [64]:
cvnn

array([ 0.11151204,  0.301639  , -0.07975545])

In [165]:
nn.score(X_test, y_test)

C:\Users\gsuri\AppData\Local\Continuum\anaconda3\envs\Python3.7\lib\site-packages\sklearn\base.py:420: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)


0.07524084799303367

In [167]:
nn2.score(X_test_final, y_test_final)


C:\Users\gsuri\AppData\Local\Continuum\anaconda3\envs\Python3.7\lib\site-packages\sklearn\base.py:420: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)


0.2626550080082924

In [31]:
dump(nn, 'relu38percent.joblib') 

['relu38percent.joblib']

In [ ]:
# Adding Random Forest to combine the two models:


12:35 - 0.5231418465059838

Adam  - 0.25864671144762147

# Pytorch NN

In [56]:
class Net(torch.nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.hidden1 = torch.nn.Linear(300, 512)
        self.hidden2 = torch.nn.Linear(512, 1024)
        self.predict = torch.nn.Linear(1024, 1000)

    def forward(self, x):
        x = F.relu(self.hidden1(x))
        x = F.relu(self.hidden2(x))
        x = self.predict(x) 
        return x


net = Net()
print(net)

optimizer = torch.optim.Adam(net.parameters(), lr=0.01)
loss_func = torch.nn.MSELoss()

BATCH_SIZE = 64
EPOCH = 200

torch_dataset = Data.TensorDataset(torch.from_numpy(X_train), torch.from_numpy(y_train))

loader = Data.DataLoader(
    dataset=torch_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=True, num_workers=2,)

Net(
  (hidden1): Linear(in_features=300, out_features=512, bias=True)
  (hidden2): Linear(in_features=512, out_features=1024, bias=True)
  (predict): Linear(in_features=1024, out_features=1000, bias=True)
)


In [57]:
for epoch in range(EPOCH):
    
    for step, (batch_x, batch_y) in enumerate(loader):

        b_x = Variable(batch_x)
        b_y = Variable(batch_y)

        prediction = net(b_x)

        loss = loss_func(prediction, b_y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

RuntimeError: Expected object of scalar type Float but got scalar type Double for argument #2 'mat1' in call to _th_addmm

# Model Validation

In [90]:
model = nn

In [91]:
def score_prediction(true, pred):

    if true not in pred:
        return 0.0
    
    ind = pred.index(true)
    
    return (20 - ind) / 20

In [92]:
def nearest_neighbors(sample, targets):
    """Returns index positions of nearest neighbors"""
    distances = [np.linalg.norm(t-sample) for t in targets]
    distances_sorted = np.argsort(distances)
    return distances_sorted[:20]

In [93]:
predicted_fvs = [model.predict(desc.reshape(1, -1)) for desc in X_test]
pred_indices = [nearest_neighbors(sample, y_test) for sample in predicted_fvs]

preds = [
    [y_true[ind] for ind in pred]
    for pred in pred_indices
]

# score = sum([1 if y_true[ind] in pred else 0 for ind, pred in enumerate(preds)]) / len(preds)

In [94]:
score = sum(
    [score_prediction(y_true[ind], pred) for ind, pred in enumerate(preds)]
           ) / len(preds)

In [95]:
print(score)

0.3927749999999999


12:35 score - 0.37332099999999513

random forest - 0.0018503043118560125

12/10 score - 0.39

Ridge - 0.0998

Kernel ridge - 0.29

# Making predictions

In [96]:
model = nn

In [97]:
mlb_tags_train2, mlb_tags_test2, df_test = dp.load_dataframe(train_or_test='test')

df_test['desc_joined'] = df_test['descriptions'].apply(lambda x: " ".join(x).split(" "))

In [98]:
df_test.head()

,image_file,resnet_vector,descriptions,tags,tags_values,tags_vec,word_list,word_vector,desc_joined
0,images_test/152.jpg,"[-0.148756742477417, -0.4813389182090759, -0.6...","[A sign sitting above a store front entrance.,...",[person:person],[person],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[A, sign, sit, store, front, entranc, small, a...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[A, sign, sitting, above, a, store, front, ent..."
1,images_test/901.jpg,"[-2.7743351459503174, -2.609677791595459, -2.2...",[A steam locamotive passes by some houses on a...,[indoor:clock],[clock],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[A, steam, locamot, pass, hous, track, A, smal...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[A, steam, locamotive, passes, by, some, house..."
2,images_test/1609.jpg,"[-4.7030134201049805, -3.1210086345672607, -0....",[a woman with purple hair is taking a picture ...,"[animal:dog, animal:horse, person:person]","[dog, hors, person]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[woman, purpl, hair, take, pictur, A, hipster,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[a, woman, with, purple, hair, is, taking, a, ..."
3,images_test/501.jpg,"[-4.7203192710876465, -3.283935546875, -3.4253...",[A GREEN AND BROWN SUITCASE LYING UP ON A LEDG...,[furniture:toilet],[toilet],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[A, green, and, brown, suitcas, ly, UP, ON, A,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[A, GREEN, AND, BROWN, SUITCASE, LYING, UP, ON..."
4,images_test/517.jpg,"[-2.5421407222747803, 0.7139833569526672, -4.4...",[A desk with a small white computer set up on ...,"[animal:cat, kitchen:bowl, furniture:bed]","[cat, bowl, bed]","[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, ...","[A, desk, small, white, comput, set, A, desk, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[A, desk, with, a, small, white, computer, set..."


In [175]:
test_descriptions = get_word2vec(df_test, w2v_model)

y_resnet = np.asarray([int_features_test[x] for x in df_test['image_file']], dtype=np.float64)
y_targets = df_test['image_file'].to_numpy()

In [176]:
predicted_fvs = [model.predict(desc.reshape(1, -1)) for desc in test_descriptions]
pred_indices = [nearest_neighbors(sample, y_resnet) for sample in predicted_fvs]

preds = [
    [y_targets[ind] for ind in pred]
    for pred in pred_indices
]

preds = [[file.split('/')[-1] for file in pred] for pred in preds]

In [177]:
desc_txt_files = df_test['image_file'].apply(
        lambda x: x.split('/')[-1].replace('jpg', 'txt')
    )

In [178]:
out_preds = list(zip(desc_txt_files.to_numpy(), preds))

In [111]:
with open('nn.csv', 'w+') as f:
    f.write('Descritpion_ID,Top_20_Image_IDs\n')
    for pred in out_preds:
        f.write(f"{pred[0]},{' '.join(pred[1])}\n")

In [17]:
#VOTING MOCLUCKIN CLASSIFIER

In [168]:
from sklearn.ensemble import VotingRegressor

kr = KernelRidge(kernel=RBF(1))
kr1 = KernelRidge()
lr = LinearRegression()

er = VotingRegressor([('KernelRidge', kr1), ('rf', kr)])


In [173]:
er.fit(X_train_final, y_train_final)

VotingRegressor(estimators=[('KernelRidge',
                             KernelRidge(alpha=1, coef0=1, degree=3, gamma=None,
                                         kernel='linear', kernel_params=None)),
                            ('rf',
                             KernelRidge(alpha=1, coef0=1, degree=3, gamma=None,
                                         kernel=RBF(length_scale=1),
                                         kernel_params=None))],
                n_jobs=None, weights=None)

In [46]:
print(X_train.shape)
print(y_train.shape)


(8000, 300)
(8000, 2048)


In [171]:
print(X_test.shape)
print(y_test.shape)
er.predict(X_test)


(2000, 300)
(2000, 2048)


array([[0.56334416, 0.56151411],
       [0.51961793, 0.5137668 ],
       [0.35565083, 0.3539893 ],
       ...,
       [0.29017524, 0.29018393],
       [0.44395211, 0.44398691],
       [0.33304492, 0.33172155]])

In [174]:
er.score(X_test_final, y_test_final)

ValueError: Found input variables with inconsistent numbers of samples: [2000, 1000]

In [180]:
def use_model_to_predict(nn, features_test):
    
    mlb_tags_train2, mlb_tags_test2, df_test = dp.load_dataframe(train_or_test='test')

    df_test['desc_joined'] = df_test['descriptions'].apply(lambda x: " ".join(x).split(" "))
    test_descriptions = get_word2vec(df_test, w2v_model)

    y_resnet = np.asarray([features_test[x] for x in df_test['image_file']], dtype=np.float64)
    y_targets = df_test['image_file'].to_numpy()

    preds = [
        [y_targets[ind] for ind in pred]
        for pred in pred_indices
    ]

    preds = [[file.split('/')[-1] for file in pred] for pred in preds]
    desc_txt_files = df_test['image_file'].apply(
            lambda x: x.split('/')[-1].replace('jpg', 'txt')
        )
    out_preds = list(zip(desc_txt_files.to_numpy(), preds))
    return out_preds

out_preds1 = use_model_to_predict(nn, features_test)
out_preds2 = use_model_to_predict(nn2, int_features_test)




AttributeError: 'dict' object has no attribute 'shape'

In [186]:
# print(features_test)
# print(int_features_test.shape)
len(features_test['images_test/152.jpg'])


1000

In [189]:
# out_preds1 = np.asmatrix(out_preds1).ravel()
# print(out_preds1)


max_found= float('-inf')
for i, tup in enumerate(out_preds1):
    print(tup)
#     print(np.intersect1d(out_preds1[i][1], out_preds2[i][1], return_indices = True))
    print(out_preds2[i][1])
#     max_found = max(len(np.intersect1d(out_preds1[i][1], out_preds2[i][1], return_indices=True)), max_found)
max_found

print(out_preds1==out_preds2)


('152.txt', ['1344.jpg', '1475.jpg', '1482.jpg', '1413.jpg', '663.jpg', '871.jpg', '1119.jpg', '1291.jpg', '979.jpg', '1980.jpg', '1942.jpg', '868.jpg', '393.jpg', '1040.jpg', '633.jpg', '821.jpg', '1919.jpg', '609.jpg', '1648.jpg', '1553.jpg'])
['1344.jpg', '1475.jpg', '1482.jpg', '1413.jpg', '663.jpg', '871.jpg', '1119.jpg', '1291.jpg', '979.jpg', '1980.jpg', '1942.jpg', '868.jpg', '393.jpg', '1040.jpg', '633.jpg', '821.jpg', '1919.jpg', '609.jpg', '1648.jpg', '1553.jpg']
('901.txt', ['248.jpg', '1583.jpg', '277.jpg', '805.jpg', '1387.jpg', '283.jpg', '1766.jpg', '1884.jpg', '1501.jpg', '1666.jpg', '551.jpg', '284.jpg', '710.jpg', '1594.jpg', '476.jpg', '195.jpg', '800.jpg', '923.jpg', '1630.jpg', '1397.jpg'])
['248.jpg', '1583.jpg', '277.jpg', '805.jpg', '1387.jpg', '283.jpg', '1766.jpg', '1884.jpg', '1501.jpg', '1666.jpg', '551.jpg', '284.jpg', '710.jpg', '1594.jpg', '476.jpg', '195.jpg', '800.jpg', '923.jpg', '1630.jpg', '1397.jpg']
('1609.txt', ['21.jpg', '484.jpg', '1131.jpg', '

('265.txt', ['780.jpg', '1490.jpg', '1356.jpg', '1422.jpg', '1554.jpg', '1873.jpg', '733.jpg', '39.jpg', '578.jpg', '924.jpg', '1113.jpg', '1058.jpg', '1964.jpg', '34.jpg', '827.jpg', '324.jpg', '1545.jpg', '556.jpg', '1805.jpg', '1007.jpg'])
['780.jpg', '1490.jpg', '1356.jpg', '1422.jpg', '1554.jpg', '1873.jpg', '733.jpg', '39.jpg', '578.jpg', '924.jpg', '1113.jpg', '1058.jpg', '1964.jpg', '34.jpg', '827.jpg', '324.jpg', '1545.jpg', '556.jpg', '1805.jpg', '1007.jpg']
('1537.txt', ['1807.jpg', '482.jpg', '1068.jpg', '1714.jpg', '538.jpg', '121.jpg', '1733.jpg', '1199.jpg', '1184.jpg', '818.jpg', '571.jpg', '1989.jpg', '1408.jpg', '589.jpg', '378.jpg', '504.jpg', '674.jpg', '1143.jpg', '738.jpg', '1477.jpg'])
['1807.jpg', '482.jpg', '1068.jpg', '1714.jpg', '538.jpg', '121.jpg', '1733.jpg', '1199.jpg', '1184.jpg', '818.jpg', '571.jpg', '1989.jpg', '1408.jpg', '589.jpg', '378.jpg', '504.jpg', '674.jpg', '1143.jpg', '738.jpg', '1477.jpg']
('1863.txt', ['788.jpg', '1402.jpg', '164.jpg', '74

In [142]:
final

matrix([['152.txt',
         list(['1344.jpg', '1475.jpg', '1482.jpg', '1413.jpg', '663.jpg', '871.jpg', '1119.jpg', '1291.jpg', '979.jpg', '1980.jpg', '1942.jpg', '868.jpg', '393.jpg', '1040.jpg', '633.jpg', '821.jpg', '1919.jpg', '609.jpg', '1648.jpg', '1553.jpg']),
         '901.txt', ...,
         list(['484.jpg', '425.jpg', '1086.jpg', '946.jpg', '911.jpg', '395.jpg', '1608.jpg', '1658.jpg', '1699.jpg', '1170.jpg', '84.jpg', '574.jpg', '312.jpg', '108.jpg', '308.jpg', '1131.jpg', '1535.jpg', '1743.jpg', '1942.jpg', '292.jpg']),
         '1164.txt',
         list(['1271.jpg', '548.jpg', '435.jpg', '1986.jpg', '1958.jpg', '816.jpg', '596.jpg', '1293.jpg', '462.jpg', '58.jpg', '1182.jpg', '1498.jpg', '424.jpg', '293.jpg', '108.jpg', '1531.jpg', '1975.jpg', '782.jpg', '1359.jpg', '1163.jpg'])]],
       dtype=object)